In [1]:
!nvidia-smi

Tue Mar  4 17:14:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   49C    P0             14W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [2]:
import os
HOME = os.getcwd()
print(HOME)

e:\Fault Detection System


In [3]:
!pip install roboflow


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import os

# Define HOME explicitly
HOME = "e:/Fault Detection System"

# Create directory
os.makedirs(f"{HOME}/datasets", exist_ok=True)

# Change directory
os.chdir(f"{HOME}/datasets")

# Now import and use roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="pDQl3KzojvPFDPZBHzlN")
project = rf.workspace("test1-trmor").project("product-printing-fault-detection")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Product-Printing-Fault-Detection-1 in yolov11:: 100%|██████████| 1112/1112 [00:00<00:00, 1505.81it/s]


## Custom Training

In [5]:
!pip install ultralytics


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Install ultralytics (run this in a separate cell first)
!pip install ultralytics

# Main code (run this after installation succeeds)
import os
from ultralytics import YOLO

# Define HOME explicitly
HOME = "e:/Fault Detection System"

# Change directory
os.chdir(HOME)

# Load and train the nano model
model = YOLO("yolo11n.pt")  # Changed from yolo11s.pt to yolo11n.pt
model.train(data=f"{dataset.location}/data.yaml", epochs=30, imgsz=480, plots=True)

**NOTE:** The results of the completed training are saved in `{HOME}/runs/detect/train/`. Let's examine them.

In [ ]:
!ls {HOME}/runs/detect/train2/

In [ ]:
from IPython.display import Image as IPyImage
HOME = "e:/Fault Detection System"
IPyImage(filename=f'{HOME}/runs/detect/train2/confusion_matrix.png', width=600)

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/train2/results.png', width=600)

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/train2/val_batch0_pred.jpg', width=600)

## Validate fine-tuned model

In [ ]:
!yolo task=detect mode=val model={HOME}/runs/detect/train2/weights/best.pt data={dataset.location}/data.yaml

## Inference with custom model

In [ ]:
!yolo task=detect mode=predict model={HOME}/runs/detect/train2/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

**NOTE:** Let's take a look at few results.

In [ ]:
import glob
import os
from IPython.display import Image as IPyImage, display

HOME = os.path.expanduser("~")  # Define the HOME variable
folders = glob.glob(f'{HOME}/runs/detect/predict*/')
if not folders:
    raise ValueError("No folders found matching the pattern.")
latest_folder = max(folders, key=os.path.getmtime)
for img in glob.glob(f'{latest_folder}/*.jpg')[:3]:
    display(IPyImage(filename=img, width=600))
    print("\n")